In [7]:
import os
import sys
import json
import numpy as np
import copy
import time

In [14]:
sentences = [ np.delete(i[1],i[0]).tolist() for i in  json.load(open('../data/feature_extraction/entity_pair_sentences.json'))]
standart_sentences = [ np.delete(i[1],i[0]).tolist() for i in  json.load(open('../data/feature_extraction/entity_pair_sentences.json'))]

In [24]:
begin = time.time()
res = {}
buf = copy.copy(sentences)
track = 0
for file in os.listdir('../data/preprocessing/abstract_extract_filtered_tracked/') :
    data = json.load(open('../data/preprocessing/abstract_extract_filtered_tracked/%s'%file))
    print('check file %s'%file)
    for ID in data :
        ref = data[ID]
        for sentence in sentences :
            thresh_hold = 0
            check = 0
            success = 0
            for word in sentence:
                if word in ref :
                    check+=1
                elif thresh_hold == 0 :
                    thresh_hold = 1
                else :
                    break
                if (check + thresh_hold) == len(sentence) :
                    res[standart_sentences.index(sentence)] = [file,ID,data[ID]]
                    del buf[sentences.index(sentence)]
                    success = 1
                    track +=1
            if success == 1:
                break
        sentences = copy.copy(buf)
    
    print('%s / 31917'%track)

print(time.time() - begin)

check file abtract_dict_1.json
466 / 31917
check file abtract_dict_10.json
622 / 31917
check file abtract_dict_100.json
626 / 31917
check file abtract_dict_101.json
630 / 31917
check file abtract_dict_102.json
663 / 31917
check file abtract_dict_103.json
663 / 31917
check file abtract_dict_104.json
668 / 31917
check file abtract_dict_105.json
669 / 31917
check file abtract_dict_106.json
673 / 31917
check file abtract_dict_107.json
675 / 31917
check file abtract_dict_108.json
678 / 31917
check file abtract_dict_109.json
680 / 31917
check file abtract_dict_11.json
767 / 31917
check file abtract_dict_110.json
768 / 31917
check file abtract_dict_111.json
770 / 31917
check file abtract_dict_112.json
772 / 31917
check file abtract_dict_113.json
810 / 31917
check file abtract_dict_114.json
829 / 31917
check file abtract_dict_115.json
832 / 31917
check file abtract_dict_116.json
858 / 31917
check file abtract_dict_117.json
924 / 31917
check file abtract_dict_118.json
967 / 31917
check file abt

KeyboardInterrupt: 

In [28]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from textblob import TextBlob
import nltk
import json
import os
import sys
import time
import operator
#import re
import string
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
ps = PorterStemmer()

In [29]:
########### Trigger word
triggers_raw = open('..//data//preprocessing/triggers.txt').read().split('\n')[:-1]
triggers = []
for row in triggers_raw :
    triggers.append(ps.stem(row.split('\t')[-1]))
    
triggers = list(set(triggers))

In [30]:
#### Drug synonyms from drugbank
#drugs = json.load(open('..//data/preprocessing/drug_corpus/drug_name_synonyms.json',encoding='utf-8'))
drugs = json.load(open('..//data/preprocessing/drug_corpus/drugs.json',encoding='utf-8'))
#### Herb scientific name and its synonyms (not alias)
#herbs = json.load(open('..//data/preprocessing/herb_corpus/herb_science_synonyms.json',encoding='utf-8'))
herbs = json.load(open('..//data/preprocessing/herb_corpus/herb_syms_total.json',encoding='utf-8'))
#### standard ref without effect
ref_without = json.load(open('..//data/preprocessing/standard_ref/standard_ref_without_effect.json',encoding='utf-8'))

In [31]:
###### Drug entity
drug_entity = {}
drugorder = 0
for ID in drugs :
    for drug in drugs[ID] :
        if drug.lower() in herbs or drug.lower() in ref_without :
            break
        drug_entity[drug.lower()] = 'drug_entity_%s'%drugorder
    drugorder+=1

###### Herb entity 
herb_entity = {}
herborder = 0
for herb in herbs :
    #herb_entity[herb.lower()] = 'herb_entity_%s'%herborder
    for synonym in herbs[herb] :
        herb_entity[synonym.lower()] = 'herb_entity_%s'%herborder
    herborder+=1
    
    
###### standard ref without effect entity 
for herb in ref_without :
    if herb.lower() not in herb_entity :
        herb_entity[herb.lower()] = 'herb_entity_%s'%herborder
        herborder+=1
    for drug in ref_without[herb] :
        if drug.lower() not in drug_entity :
            drug_entity[drug.lower()] = 'drug_entity_%s'%drugorder
            drugorder+=1

In [32]:
def entity_replace(text) :
    ###
    begin = time.time()
    ###
    for herb in herb_entity :
        if herb in text :
            text = text.replace(herb,' %s '%herb_entity[herb])

    ###
    drug_entity_single = {}
    for drug in drug_entity :
        if drug in text :
            text = text.replace(drug,' %s '%drug_entity[drug])




    return text

In [33]:
def word_processing(text,res,file,ID,stop_words,punctuations,data) :
    #######
    check = 0
    
    ###### Split text into words
    sentences = [ x for x in TextBlob(text).sentences if len(x.split()) > 2 ]
    ########
    sentence_noun_verb = []
    ###### Processing
    for sentence in sentences :
        #######
        words = sentence.tags
        trigger_check = 0
        for word in words : 
            ###### Full sentence
            if word[0] not in stop_words and word[0] not in punctuations :
                if ps.stem(word[0]) in triggers :
                    trigger_check = 1
                ###### Pos tagging
                if word[1][0] == 'N' or word[1][0] == 'V'  :
                    sentence_noun_verb.append(ps.stem(word[0]))

                    
        ###### Check point
        if trigger_check == 1 :
            if len(sentence_noun_verb) > 3 :
                if sentence_noun_verb in data :
                    res[data.index(sentence_noun_verb)] = [file,ID]
                    check = 1
        
        
        ##### Reset
        sentence_noun_verb = []
        if check == 1 :
            break
        ######

    return res

In [39]:
########### Text source
stop_words = list(set(stopwords.words('english')))
punctuations = string.punctuation
###########
text_list = os.listdir('..//data/preprocessing/abstract_extract_filtered_tracked//')[:232]
count = 0
textdict = []
res = {}
data = [ i[1] for i in json.load(open('../data/feature_extraction/entity_pair_sentences.json'))]
    
for file in text_list :
    textdict = json.load(open('..//data//preprocessing/abstract_extract_filtered_tracked/%s'%file))
    #text = rmv_n(textdict)
    #textdict = []
    #####
    #print('step 1')
    for ID in textdict :
        text = ''.join(textdict[ID].split('\n'))
        text = entity_replace(text)
        #####
        #print('step 2')
        res = word_processing(text,res,file,ID,stop_words,punctuations,data)
        text = ''
    #####
    ##### 
    print('%s / 31917'%len(res))
    print('%s done'%(file))

#########
outdir = '..//data//'
outfile = open(os.path.join(outdir,os.path.basename('sen_index_1.json')),'w')
outfile.write(json.dumps(res))
outfile.close()

4 / 31917
abtract_dict_1.json done
9 / 31917
abtract_dict_10.json done
9 / 31917
abtract_dict_100.json done
9 / 31917
abtract_dict_101.json done
9 / 31917
abtract_dict_102.json done
9 / 31917
abtract_dict_103.json done
10 / 31917
abtract_dict_104.json done
10 / 31917
abtract_dict_105.json done
10 / 31917
abtract_dict_106.json done
10 / 31917
abtract_dict_107.json done
11 / 31917
abtract_dict_108.json done
12 / 31917
abtract_dict_109.json done
13 / 31917
abtract_dict_11.json done
13 / 31917
abtract_dict_110.json done
13 / 31917
abtract_dict_111.json done
13 / 31917
abtract_dict_112.json done
17 / 31917
abtract_dict_113.json done
19 / 31917
abtract_dict_114.json done
20 / 31917
abtract_dict_115.json done
20 / 31917
abtract_dict_116.json done
27 / 31917
abtract_dict_117.json done
29 / 31917
abtract_dict_118.json done
31 / 31917
abtract_dict_119.json done
41 / 31917
abtract_dict_12.json done
45 / 31917
abtract_dict_120.json done
50 / 31917
abtract_dict_121.json done
54 / 31917
abtract_dict

In [38]:
(text_list[:232] + text_list[232:465] + text_list[465:696] +text_list[696:] ) == text_list

True